<a href="https://colab.research.google.com/github/AmoghUpadhayay/Machine-Learning-Projects/blob/main/Facial_Emotions_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 16.07 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Checking out files: 100% (13996/13996), done.


In [ ]:
import csv
data={}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader=csv.reader(f)
  next(reader)
  for row in reader:
    key=row[2].lower()
    if(key in data):
      data[key].append(row[1])
    else:
      data[key]=[row[1]]

In [ ]:
emotion_list=list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [ ]:
import os
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [ ]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training/',emotion))
  os.mkdir(os.path.join('master_data/testing/',emotion))

In [ ]:
from shutil import copyfile
split_size=0.8

for emotion,images in data.items():
  train_size=int(split_size*len(images))
  train_images=images[:train_size]
  test_images=images[train_size:]
  for image in train_images:
    source=os.path.join('/content/facial_expressions/images',image)
    dest=os.path.join('/content/master_data/training',emotion,image)
    copyfile(source,dest)
  for image in test_images:
    source=os.path.join('/content/facial_expressions/images',image)
    dest=os.path.join('/content/master_data/testing',emotion,image)
    copyfile(source,dest)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model=Sequential([
                  Conv2D(16,kernel_size=(3,3),activation='relu',input_shape=(100,100,3)),
                  MaxPooling2D(2,2),
                  Conv2D(32,kernel_size=(3,3),activation='relu'),
                  MaxPooling2D(2,2),
                  Conv2D(64,kernel_size=(3,3),activation='relu'),
                  MaxPooling2D(2,2),
                  Flatten()
                  ,Dense(1024,activation='relu')
                  ,Dense(8,activation='softmax')
                  ])

In [ ]:
model.compile(optimizer = Adam(lr=0.001),loss = 'categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 10, 64)       0

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_dir='/content/master_data/training'
test_dir='/content/master_data/testing'

train_gen=ImageDataGenerator(rescale=1.0/255)
train_data=train_gen.flow_from_directory(
    train_dir,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=128
    )

test_gen=ImageDataGenerator(rescale=1.0/255)
test_data=test_gen.flow_from_directory(
    test_dir,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=128
    )

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [ ]:
es=EarlyStopping(monitor='accuracy',patience=2,min_delta=0.01)

In [ ]:
model.fit_generator(train_data,
          epochs=10,
          verbose=1,
          validation_data=test_data,
          callbacks=[es])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/10
86/86 [==============================] - 120s 1s/step - loss: 0.9387 - accuracy: 0.6113 - val_loss: 0.9934 - val_accuracy: 0.6331
Epoch 2/10
86/86 [==============================] - 116s 1s/step - loss: 0.5971 - accuracy: 0.7954 - val_loss: 0.8074 - val_accuracy: 0.7236
Epoch 3/10
86/86 [==============================] - 115s 1s/step - loss: 0.4966 - accuracy: 0.8320 - val_loss: 1.3872 - val_accuracy: 0.6813
Epoch 4/10
86/86 [==============================] - 116s 1s/step - loss: 0.4578 - accuracy: 0.8409 - val_loss: 1.1822 - val_accuracy: 0.6853
Epoch 5/10
86/86 [==============================] - 115s 1s/step - loss: 0.4194 - accuracy: 0.8567 - val_loss: 1.5339 - val_accuracy: 0.6681
Epoch 6/10
86/86 [==============================] - 116s 1s/step - loss: 0.3730 - accuracy: 0.8685 - val_loss: 1.6476 - val_accuracy: 0.6809
Epoch 7/10
86/86 [==============================] - 115s 1s/step - loss: 0.3328 - accuracy: 0.8813 - val_loss: 1.8345 - val_accuracy: 0.6685
Epoch 8/10
86

In [ ]:
from google.colab import drive
drive.mount('/content/drive')